In [1]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 

In [2]:
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

import os
from getpass import getpass


In [3]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter Mistral API Key: ")


Enter Mistral API Key: ··········


In [4]:
!pip install pypdf


In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/2022-GROUP-II_20SERVICES-NOTIFN_20FINAL2022122923474120230118102628 (1).pdf")   # upload your PDF to colab
docs = loader.load()
len(docs)


46

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(docs)
len(chunks)


300

In [7]:
embeddings = MistralAIEmbeddings(model="mistral-embed")

vector_db = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [9]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


In [11]:
template = """
You are a Memory-Augmented RAG assistant.

Conversation History:
{chat_history}

Relevant Retrieved Context:
{context}

User Query:
{question}

Provide a context-aware answer. ALWAYS use history if relevant.
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template=template
)


In [12]:
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)


In [ ]:
while True:
    q = input("You: ")
    if q.lower() in ["exit", "quit"]:
        break
    resp = rag_chain.invoke({"question": q})
    print("Bot:", resp["answer"])


You: Can you tell me what is discussd in the document
Bot: Based on the **retrieved context** and **conversation history**, the document discusses the following key points:

1. **Vacancy Break-Up for Assistant Section Officer (ASO) Posts**:
   - The document provides a detailed **category-wise and gender-wise breakdown of vacancies** for two departments:
     - **Election Commission (Post Code-14)**
     - **Legislative Secretariat (Post Code-11)**
   - The vacancies are categorized under:
     - **Reservation categories** (OC, EWS, BC-A, BC-B, BC-C, BC-D, BC-E, SC, ST)
     - **Special categories** (Sports, Physically Handicapped - PH, with subcategories like VH, HH, OH, MH)
     - **Gender** (G = General/Male, W = Women)

2. **Key Observations from the Data**:
   - **Election Commission (Post Code-14)**:
     - **Total vacancies**: 165 (109 General + 56 Women).
     - **State-wide vacancies**: Only 2 (1 General + 1 Woman).
     - **Highest vacancies**: OC (30 General, 19 Women), SC (

In [ ]:
from google.colab import drive
drive.mount('/content/drive')